In [1]:
txt = open('F:/资源/CSCMNews/科技/481676.txt',encoding='utf-8').read() #读取文件

In [2]:
from nltk import *
from matplotlib import pyplot as plt
import re,jieba,matplotlib,paddle
import jieba.posseg as psg
#正则清洗多余换行空格
txt=re.sub('\t|\n|\s|\r',' ',txt)
txt=re.sub('\s+',' ',txt)

## 1.利用词性标注进行命名实体识别

In [3]:
featWord=""
user_pos_list=['LOC','PER','ORG','TIME']#PER 人名 LOC 地名 ORG 机构名 TIME 时间
paddle.enable_static() #使用jieba的paddle模式，调用PaddlePaddle深度学习框架进行分词 相对于默认模式分词更加准确
jieba.enable_paddle()
for word,pos in psg.cut(txt,use_paddle=True):
    if pos in user_pos_list:
        if word+' '+pos+'\n'not in featWord:
            featWord+=word+' '+pos+'\n'
print(featWord)

C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Paddle enabled successfully......


著作权协会 ORG
7月30日 TIME
中国电影著作权协会 ORG
上海宽娱数码科技有限公司 ORG
杭州世纪联线网络技术有限公司 ORG
世纪联线 ORG
上海隐志网络科技有限公司 ORG
华谊兄弟 ORG
孙健红 PER
7月20日 TIME
华谊 ORG
华谊兄弟传媒股份有限公司 ORG
7月22日 TIME
朱永德 PER
北京市 LOC
上海 LOC
杭州“世纪联线 ORG
2010年7月21日 TIME
国家版权局 ORG
杭州“世纪联线” ORG
北京 LOC
协和大盛国际传媒(北京)有限公司 ORG
2010年5月4日 TIME
上海隐志网络公司 ORG
著协 ORG
上海市第一中级人民法院 ORG
中国发行放映协会 ORG
孙慧 PER



## 2.日期识别 

In [4]:
import re
from datetime import datetime,timedelta
import jieba.posseg as psg
#数字，单位映射
Numbermap={
    '零': 0, '一': 1, '二': 2, '两': 2, '三': 3, '四': 4,
    '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
    '5': 5, '6': 6, '7': 7, '8': 8, '9': 9
}

In [5]:
#解析日期
def Resolution(Word): #根据正则将文本分块
    txt = re.match(r"([0-9零一二两三四五六七八九十]+年)?([0-9一二两三四五六七八九十]+月)?([0-9一二两三四五六七八九十]+[号日])?([上中下午晚早]+)?([0-9零一二两三四五六七八九十百]+[点:\.时])?([0-9零一二三四五六七八九十百]+分?)?([0-9零一二三四五六七八九十百]+秒)?",Word)
    if txt.group(0) is not None:
        res = {
            "year": txt.group(1),
            "month": txt.group(2),
            "day": txt.group(3),
            "noon":txt.group(4),  # 上中下午晚早
            "hour": txt.group(5) if txt.group(5) is not None else '00',#为空则为00
            "minute": txt.group(6) if txt.group(6) is not None else '00',
            "second": txt.group(7) if txt.group(7) is not None else '00'
        }
        params = {}
        for key in res: #分块映射
            if res[key] is not None and len(res[key]) != 0:
                tmp = None
                if key=='year':
                    year=''
                    for item in res[key]:#遍历
                        if item in Numbermap.keys(): #将中文数字或字符串数字转换为数字格式
                            year += str(Numbermap[item])
                    tmp=year
                else:
                    num = re.match("\d+", res[key])
                    if num:#如是数字格式则不需要转换
                        tmp=num.group(0)
                    else:
                        num=''
                        for item in res[key]:#遍历
                            if item in Numbermap.keys():
                                num += str(Numbermap[item])
                        if num!='':
                            tmp=num   
                if tmp is not None:
                    params[key] = int(tmp)
        target_date = datetime.today().replace(**params)#输出字典
        if txt.group(4) is not None and txt.group(4) in ['下午','中午','晚上']: #时区
            hour = target_date.time().hour
            if hour < 12:
                target_date = target_date.replace(hour=hour + 12)
        return target_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        return None

<>:25: DeprecationWarning: invalid escape sequence \d
<>:25: DeprecationWarning: invalid escape sequence \d
<ipython-input-5-c118665321db>:25: DeprecationWarning: invalid escape sequence \d
  num = re.match("\d+", res[key])


In [6]:
#日期识别
def Dateiden(text):
    Word=""
    preposition={'今天': 0, '明天':1, '后天': 2} #datetime.timedelta参数
    for words,noun in psg.cut(text):
        if words in preposition:#根据前置条件计算日期并转换格式
            Word = (datetime.today() +timedelta(days=preposition.get(words, 0))).\
                      strftime('%Y{y}%m{m}%d{d}').format(y='年',m='月',d='日')  
        elif noun in ['m', 't']: #识别数词时间
            Word+=words
    if Word != '':
        return Resolution(Word)
    else:
        return None

In [7]:
text1 = '10月09日返校'
print(text1, Dateiden(text1), sep=':')
text2 = '10月11号上午8点不上自然语言处理课'
print(text2, Dateiden(text2), sep=':')
text3 = '明天没课'
print(text3, Dateiden(text3), sep=':')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Taki\AppData\Local\Temp\jieba.cache
Loading model cost 0.684 seconds.
Prefix dict has been built successfully.


10月09日返校:2021-10-09 00:00:00
10月11号上午8点不上自然语言处理课:2021-10-11 08:00:00
明天没课:2021-10-19 00:00:00
明天没课但后天有课:2021-10-20 00:00:00
